In [229]:
import pandas as pd
import os
import re
import itertools
import random
import importlib
import sys
import tqdm
from tqdm.auto import tqdm
tqdm.pandas()

In [230]:
parrent_dir = os.path.dirname(os.getcwd())
sys.path.append(parrent_dir)

In [231]:
import numpy as np
from rhyme_detection.word_spectral import wordspectrum
from rhyme_detection.utils import check_rhyme



In [558]:
fpath = 'data/raw/rhymes_clean.csv'

rhyme_df = pd.read_csv(fpath)


In [582]:
rhyme_df

,word1,word2,rhyme
0,reisen,besserung,0
1,drachen,lachen,1
2,throne,geschaut,0
3,verspricht,stern,0
4,könig,unterthänig,1
...,...,...,...
36623,reine,ab,0
36624,geht,steht,1
36625,schatten,zweige,0
36626,erfüll,an,0


In [560]:
def check_duplicates(df):
    
    pair_lst = []
    idx = []
    for index,row in tqdm(df.iterrows()):
        if [row['word1'],row['word2']] not in pair_lst and [row['word2'],row['word1']] not in pair_lst:
            
            pair_lst.append([row['word1'],row['word2']])
        else:
            idx.append(index)

    return(idx)

In [584]:
duplicates = check_duplicates(rhyme_df)

0it [00:00, ?it/s]

In [585]:
rhyme_df_unique = rhyme_df.drop(duplicates)

rhyme_df_unique

,word1,word2,rhyme
0,reisen,besserung,0
1,drachen,lachen,1
2,throne,geschaut,0
3,verspricht,stern,0
4,könig,unterthänig,1
...,...,...,...
36622,riff,halcyonen,0
36623,reine,ab,0
36625,schatten,zweige,0
36626,erfüll,an,0


In [563]:
rhyme_df_sorted = rhyme_df_unique.sort_values(by=['rhyme'])

In [564]:
rhyme_df_sorted

,word1,word2,rhyme
0,reisen,besserung,0
21129,dienen,durchstikt,0
21127,bade,gar,0
21125,lippentwir,wunderbahr,0
21124,ist,seyn,0
...,...,...,...
25895,überwinden,finden,1
5903,zeit,zärtlichkeit,1
25897,finger,dinger,1
5902,hangts,schwankts,1


In [488]:
split = False

In [565]:
def shuffle_last_n(string, n=[3,6]):
    n = random.choice(n)
    last = list(string[-n:])
    first = string[:-n]
    random.shuffle(last)
    last = ''.join(last)
    
    return first + last

## if split

In [292]:
rhyme_df_sorted

,word1,word2,rhyme
0,zwingen,gewi,0
24599,vergessen,trümmern,0
24598,geschellt,krankenzimmer,0
24597,vermögen,raub,0
24595,creutz,ehrenpreisse,0
...,...,...,...
26123,engelreiz,kreuz,1
26138,purpurgewand,hand,1
9183,gier,wir,1
26130,berühren,verlieren,1


In [ ]:
if split:
    rhyme_df_head = rhyme_df_sorted.head(int(len(rhyme_df_sorted)*0.86))
    rhyme_pairs= rhyme_df_sorted.tail(int(len(rhyme_df_sorted)*0.14))
    rhyme_pairs['word2'] = rhyme_pairs['word2'].apply(lambda x: shuffle_last_n(x))
    rhyme_pairs['rhyme'] = 0

    rhyme_pairs

## if not split

In [566]:
if not split: 
    rhyme_pairs = rhyme_df_sorted.loc[rhyme_df_sorted['rhyme'] == 1]
                                                                             
    rhyme_pairs['word2'] = rhyme_pairs['word2'].apply(lambda x: shuffle_last_n(x))
    rhyme_pairs['rhyme'] = 0    
                                      
rhyme_pairs
    
    

/home/andreas/.conda/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/andreas/.conda/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,word1,word2,rhyme
3081,ackerkrume,hundmlbeeu,0
30918,metall,afhl,0
16284,drohe,rfeoh,0
3074,linden,zufnndie,0
30917,schmeissen,zerrssneei,0
...,...,...,...
25895,überwinden,ndeifn,0
5903,zeit,zärtliehcitk,0
25897,finger,nrgide,0
5902,hangts,schnwstak,0


In [491]:
def compare_words(word_1,word_2):
    spec_1 = wordspectrum(word_1)
    spec_2 = wordspectrum(word_2)

    mean, _ = check_rhyme(spec_1,spec_2,
                                        features = 'mfccs',
                                        order=0,
                                        length = 14, 
                                        cut_off = 1,
                                        min_matches=9,
                                        pool=0)
        
    return mean

In [492]:
rhyme_pairs['distance'] = rhyme_pairs.progress_apply(lambda x: compare_words(x.word1,x.word2),axis=1)

  0%|          | 0/13723 [00:00<?, ?it/s]

/home/andreas/.conda/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [493]:
rhyme_pairs

,word1,word2,rhyme,distance
43871,solte,hotle,0,8.181944
35293,rebellion,scnho,0,13.992493
42245,fulgent,coentnor,0,15.790116
5707,welt,tehl,0,8.126822
42533,erwacht,lahtc,0,24.881616
...,...,...,...,...
9096,linnen,linenn,0,5.910470
26995,aether,röterh,0,6.550041
27001,weiherspiegel,ühlge,0,7.508116
27006,gfallt,tanwdnael,0,15.852236


In [494]:
rhyme_pairs['pred'] = rhyme_pairs.apply(lambda x: x.distance > 10,axis = 1)

/home/andreas/.conda/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [495]:
#rhyme_pairs.to_csv('data/raw/synthetic_data_raw.csv',index = False)


In [567]:
rhyme_pairs = pd.read_csv('data/raw/synthetic_data_raw.csv')

In [568]:
rhyme_pairs

,word1,word2,rhyme,distance,pred
0,solte,hotle,0,8.181944,False
1,rebellion,scnho,0,13.992493,True
2,fulgent,coentnor,0,15.790116,True
3,welt,tehl,0,8.126822,False
4,erwacht,lahtc,0,24.881616,True
...,...,...,...,...,...
13718,linnen,linenn,0,5.910470,False
13719,aether,röterh,0,6.550041,False
13720,weiherspiegel,ühlge,0,7.508116,False
13721,gfallt,tanwdnael,0,15.852236,True


In [569]:
synthetic_correct = rhyme_pairs.loc[rhyme_pairs['pred'] == True]

In [570]:
synthetic_correct

,word1,word2,rhyme,distance,pred
1,rebellion,scnho,0,13.992493,True
2,fulgent,coentnor,0,15.790116,True
4,erwacht,lahtc,0,24.881616,True
5,flug,leichenguz,0,14.536348,True
6,steiget,zeietg,0,17.920660,True
...,...,...,...,...,...
13715,süsse,seüms,0,25.984519,True
13716,bringen,zerspnregni,0,18.729869,True
13717,daheime,eermi,0,12.502184,True
13721,gfallt,tanwdnael,0,15.852236,True


In [594]:
train_test_df_raw = pd.concat([rhyme_df_sorted, synthetic_correct[['word1','word2','rhyme']][:2500]], 
                        axis=0).reset_index(drop=True)

train_test_df_raw 

,word1,word2,rhyme
0,reisen,besserung,0
1,dienen,durchstikt,0
2,bade,gar,0
3,lippentwir,wunderbahr,0
4,ist,seyn,0
...,...,...,...
32325,ihn,nhi,0
32326,spitzen,ritnez,0
32327,gefangen,heimverannegl,0
32328,ab,gbar,0


In [595]:
duplicates = check_duplicates(train_test_df_raw)

0it [00:00, ?it/s]

In [603]:
train_test_df = train_test_df_raw.drop(duplicates)
train_test_df

,word1,word2,rhyme
0,reisen,besserung,0
1,dienen,durchstikt,0
2,bade,gar,0
3,lippentwir,wunderbahr,0
4,ist,seyn,0
...,...,...,...
32325,ihn,nhi,0
32326,spitzen,ritnez,0
32327,gefangen,heimverannegl,0
32328,ab,gbar,0


In [597]:
train_test_df['word1'] = train_test_df['word1'].apply(lambda x: x[::-1])
train_test_df['word2'] = train_test_df['word2'].apply(lambda x: x[::-1])

#train_test_df = train_test_df.drop_duplicates()

In [604]:
shuffled_df = train_test_df.sample(frac=1).reset_index(drop=True)


In [605]:
ratio = 0.8

df_train = shuffled_df[:int(len(shuffled_df)*ratio)]
df_val = shuffled_df[int(len(shuffled_df)*ratio):]




In [606]:
df = pd.merge(df_val, df_train, on=['word1','word2'], how='left', indicator='Exist')

df['Exist'] = np.where(df.Exist == 'both', True, False)

df.loc[df['Exist'] == True]

,word1,word2,rhyme_x,rhyme_y,Exist


In [607]:
df_val_inverted = df_val.copy()
df_val_inverted['word1'] = df_val['word2']
df_val_inverted['word2'] = df_val['word1']
df = pd.merge(df_val, df_train, on=['word1','word2'], how='left', indicator='Exist')

df['Exist'] = np.where(df.Exist == 'both', True, False)
df.loc[df['Exist'] == True]

,word1,word2,rhyme_x,rhyme_y,Exist


In [608]:
df_train.to_csv('data/train.csv',index=False)
df_val.to_csv('data/val.csv',index=False)